In [1]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [1]:
import pandas as pd
from urlparse import urlparse
import tldextract
from collections import Counter
import jieba
import json
import math
import numpy as np
from time import time

In [177]:
df = pd.read_csv('../data/clean/huizhou_web_v1.csv')

In [179]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83470 entries, 0 to 83469
Data columns (total 22 columns):
url               83470 non-null object
title             83470 non-null object
summary           83470 non-null object
content           83470 non-null object
content_seg       83470 non-null object
s_vec             83470 non-null object
domain            83470 non-null object
words_fq          83470 non-null object
w2v_centroid      83470 non-null object
content_len       83470 non-null int64
hz_w2v_sim        83470 non-null float64
hwh_w2v_sim       83470 non-null float64
hzwh_w2v_sim      83470 non-null float64
hzjz_w2v_sim      83470 non-null float64
ycjc_w2v_sim      83470 non-null float64
hzdk_w2v_sim      83470 non-null float64
hpys_w2v_sim      83470 non-null float64
hjhc_w2v_sim      83470 non-null float64
hs_w2v_sim        83470 non-null float64
xalx_w2v_sim      83470 non-null float64
hxyj_w2v_sim      83470 non-null float64
w2v_centroid_s    83470 non-null object
d

In [178]:
df.head()

,url,title,summary,content,content_seg,s_vec,domain,words_fq,w2v_centroid,content_len,...,hzwh_w2v_sim,hzjz_w2v_sim,ycjc_w2v_sim,hzdk_w2v_sim,hpys_w2v_sim,hjhc_w2v_sim,hs_w2v_sim,xalx_w2v_sim,hxyj_w2v_sim,w2v_centroid_s
0,http://cul.anhuinews.com/system/2016/11/28/007...,洪建华夫妇的竹刻人生,。不久前，我们几名政协委员应邀到正在建设中的黄山市徽派雕刻博物馆参观考察时，洪建华一边介绍新...,谁能想到，洪建华夫妇这对一无学历、二无家传、三无师承的“三无大师”成了省级非物质文化遗产·徽...,谁 能 想到 ， 洪 建华 夫妇 这 对 一 无 学历 、 二 无 家传 、 三无 师承 的...,"{u'竹木': 0.0114285714286, u'洪': 0.0571428571429...",anhuinews.com,"{u'竹木': 0.0114285714286, u'洪': 0.0571428571429...","[-0.0060236, -0.0103621, 0.00755846, -0.000569...",10703,...,0.463720,0.567314,0.573373,0.633446,0.680938,0.424617,0.311315,0.397321,0.291379,"-0.0060236,-0.0103621,0.00755846,-0.000569638,..."
1,http://cul.anhuinews.com/system/2016/11/21/007...,徽州民歌传人操明花校园授艺,。“磨米磨麦，磨米磨麦……”随着教室里传来一阵阵悦耳动听的方言民歌声，操明花正兴致勃勃地教同...,近日，省非物质文化遗产徽州民歌传承人操明花前往休宁县海阳二小、歙县城关小学和徽州区岩寺镇中心...,近日 ， 省非 物质 文化遗产 徽州 民歌 传承 人操 明花 前往 休宁县 海阳 二小 、 ...,"{u'它': 0.0123456790123, u'原生态': 0.012345679012...",anhuinews.com,"{u'它': 0.0123456790123, u'原生态': 0.012345679012...","[-0.0107781, 0.0129855, 0.00413211, 0.00171845...",1264,...,0.565720,0.358653,0.557331,0.468289,0.517698,0.438723,0.306038,0.308822,0.250730,"-0.0107781,0.0129855,0.00413211,0.00171845,-0...."
2,http://cul.anhuinews.com/system/2016/11/03/007...,黄山楹联漫谈,。黄山雄伟奇丽、气势非凡，原省文联主席赖少其说，非有大手笔，不能画黄山。有“二联”站得高，看...,黄山楹联第一人当推明朝诗人余绍祉。他遍历祖国名山大川，见到无数奇峰怪石，却感到抵不上黄山峰林...,黄山 楹联 第一 人当 推 明朝 诗人 余绍祉 。 他 遍历 祖国 名山大川 ， 见到 无数...,"{u'气派': 0.0123456790123, u'名句': 0.012345679012...",anhuinews.com,"{u'气派': 0.0123456790123, u'名句': 0.012345679012...","[-0.0120081, -0.0145887, 0.0144744, -0.0120817...",2028,...,0.255002,0.260202,0.269191,0.340901,0.296441,0.235815,0.191698,0.246864,0.182968,"-0.0120081,-0.0145887,0.0144744,-0.0120817,-0...."
3,http://cul.anhuinews.com/system/2016/01/26/007...,徽派雕刻——徽州木雕艺术,徽州民居、家具木板和圆木雕刻艺术的简称。木雕在旧属徽州各县分布之广在全国屈指可数，宅院内的屏...,徽州民居、家具木板和圆木雕刻艺术的简称。木雕在旧属徽州各县分布之广在全国屈指可数，宅院内的屏...,徽州 民居 、 家具 木板 和 圆木 雕刻 艺术 的 简称 。 木雕 在 旧属 徽州 各县 ...,"{u'吉祥图案': 0.0172413793103, u'少数': 0.0172413793...",anhuinews.com,"{u'吉祥图案': 0.0172413793103, u'少数': 0.0172413793...","[-0.0134407, -0.0106611, 0.0100197, -0.0117281...",721,...,0.463754,0.588129,0.565690,0.765616,0.544592,0.325793,0.382593,0.354244,0.263324,"-0.0134407,-0.0106611,0.0100197,-0.0117281,-0...."
4,http://cul.anhuinews.com/system/2016/01/26/007...,徽派雕刻——徽州砖雕艺术,。从早期的简单粗犷、朴素的纹样，逐步演变到后期的繁腹、细致华丽的结构，形成了自己独特的艺术风...,徽州砖雕徽州砖雕的造型，来源于汉画象砖。汉画象砖大都是模印砖坯刻画后入密烧制，嵌窃时再进行修...,徽州 砖雕 徽州 砖雕 的 造型 ， 来源于 汉 画象 砖 。 汉 画象 砖 大都 是 模印...,"{u'一': 0.0106382978723, u'它': 0.0212765957447,...",anhuinews.com,"{u'一': 0.0106382978723, u'它': 0.0212765957447,...","[-0.0147771, -0.0149903, 0.0132143, -0.0146898...",2852,...,0.426712,0.555901,0.473556,0.605832,0.577255,0.312113,0.289483,0.315999,0.203333,"-0.0147771,-0.0149903,0.0132143,-0.0146898,-0...."


In [4]:
for i in range(0, 2000):
    doc = df.iloc[i]['content']
    sentences = doc.split()

    print i, len(sentences)
    print doc
    print '----------------'
    
    sentences = doc.split()

0 16
谁能想到，洪建华夫妇这对一无学历、二无家传、三无师承的“三无大师”成了省级非物质文化遗产·徽派竹雕传承人；谁又能想到，他们投资建设的徽派雕刻博物馆竟然成为国家级非物质文化遗产保护研究基地和国家级非物质文化遗产·徽派竹雕项目保护单位。

不久前，我们几名政协委员应邀到正在建设中的黄山市徽派雕刻博物馆参观考察时，洪建华一边介绍新馆规划布局，一边回忆他和妻子张红云的雕刻人生。

地处205国道边的这座新馆独特的景观设计，加上徽式的建设风格确实让人眼睛一亮。洪建华介绍说，新馆占地16亩、总建筑面积达1万多平方米，明年6月开馆后，人们在这里可以看到明代到民国期间徽州的竹木砖石雕的作品，还可以看到市内外当代各类著名雕刻大师的精品力作；这里不仅有他们夫妇俩的作品展示，还设有工人们的手工操作间和游客体验中心。通过实物展示和手工体验，力求让更多的人了解徽州雕刻，让更多的人了解徽州文化的独特魅力，从而更好地宣传博大精深的徽文化。

今年45岁的洪建华，中等身材，一头浓密的黑发衬着清秀的五官，笑起来还带着几分腼腆，说起竹刻文化还真是如数家珍……站在对面，让人很难把他同“安徽省非物质文化遗产·徽派竹刻传承人”、故宫自上世纪50年代以来首次收藏的现代民间竹刻笔筒“竹林七贤”的作者身份以及“全国技术能手”、国务院特殊津贴享受者等诸多头衔联系在一起。

他的老家徽州区洪坑村是一个徽文化底蕴浓厚的古村落。村中有四座古祠堂、七座古牌坊和一座状元厅，明、清风格的古民居更是鳞次栉比。这里随处可见的精美石雕和木雕，让他迷上了雕刻。少年时的他就能找些木桩竹脑雕走兽刻飞禽。初中毕业后，他经常跑到屯溪老街上去看雕刻作品。一次，他在老街买下了一本改变命运的“宝书”——文物鉴赏大家王世襄写的《竹刻》。通过学习，他开拓了眼界，提高了技艺。

因为竹刻，给洪建华带来了成功的喜悦，更因为竹刻，给他带来了很多朋友，包括他的妻子张红云。上世纪90年代初，洪建华认识了当时同在一个厂里做竹刻的张红云。两颗年轻的心因为共同的志趣走到了一起，于是两人携手开始了他们的雕刻人生。

婚后，他们有了自己的小作坊。不久，洪建华的第一件作品“八仙过海”笔筒卖了300元。这给了他极大的鼓舞，说明他的创作得到了认可，此后，他们的精品力作如“竹林七贤”“商旅图”“西厢记”“圣人泛舟”等如雨后春笋般横空出世，并收获了广泛的赞誉。2007年，

233 12
婺源茶院朱氏第九世孙朱熹，是我国南宋著名的哲学家和教育家，其一生登第五十年，“仕于外者仅九考，立朝才四十日。”在这不算长的仕途生涯中，朱熹不仅自己过着清贫俭朴的生活，常常“豆饭藜羹”；而且为官清廉，关心民间疾苦。为反对横征暴敛，他曾多次上疏“触怒皇上”，因此仕途起伏浮沉。

南宋淳熙八年，浙东一带天久不雨，水源枯竭，农田龟裂，大旱造成严重饥荒。是年八月，刚从知南康军任满的朱熹，移官提举浙东常平茶盐公事。出于对灾民的关心，朱熹“即日单车就道”。到职后，为解饥民倒悬之苦，他“极力讲求荒政”，迅即采取了几项有力措施：一是“初拜命，即移书他郡，募米商、蠲其征，及其至，客舟米已辐辏”。通过采用这种减免税收的办法，招引各地粮商来灾区做粮食生意。二是当年“十一月己亥奏事延和殿”，“上条陈七”，对孝宗临朝二十年间灾害频仍，“水旱盗贼略无宁岁”的原因及解决办法，提出了中肯的批评和建议。三是常常四处察灾情，访民隐，“视事各郡，对有不伏赈粜、不恤荒者者，皆按劾之”。并严肃查处各地有行贿受贿、徇私枉法行为的官吏。所到之处，深得民心。

次年七月中旬，朱熹巡至台州境内，见台州灾情甚重，饿殍遍野，民不聊生，饥民纷纷外流。经过查访，朱熹了解到，这是由于原台州知府、时已宣布升迁江西提刑的唐仲友，因在荒年违法促限催税而造成的。于是，朱熹对唐仲友的所作所为进行了全面调查，民怨沸腾的百姓纷纷前来告状，所列举的罪状有“促限催税，违法扰民，贪污淫虐，畜养亡命，偷盗官钱，伪造官会等事”，确是五毒俱全。为此，朱熹于七月十九日至九月上旬，先后六次给孝宗皇帝上奏状，弹劾唐仲友，要求严惩这样的贪官污吏。但因为当朝宰相王淮既是唐仲友的婺州同乡，又是他的姻亲，所以王淮等一批权贵竭力包庇唐仲友，“匿不以闻”。可经朱熹一再给皇上奏劾，且所列奏状有理有据，不得已，朝廷只得罢去了唐仲友江西提刑的新任。

在弹劾唐仲友的过程中，朱熹一身正气，不畏权势敢碰硬，表现了其崇高的操守和气节。后来，虽然朱熹因弹劾唐仲友遭到王淮一伙的诬陷，让他回家闲住，但却受到了正直人士的称颂。时太学中有人写诗道：“周公大圣犹遭谤，元晦真贤亦被讥。堪叹古今两陈贾，如何专把圣贤非。”

绍熙元年，朱熹受命出任闽南漳州知州。漳州向以民风淳厚著称，但后来由于教化不利，地方官吏行奸作恶，富豪巨室横行不法，以及经界不得推行而造成的种种弊端，使古朴的民风荡

《金瓶梅词话》第38回写到：“第二的不知高低，气不愤，走这里放水。”“放水”《金瓶梅词典》注释是“犹言发火”，发脾气。北人讳言火字，如称“失火”为“走水”。《瓶外卮言》释为：“即捣乱之意，俗语也，放水本为田家偷开坝闸之谓，利于已不利于人，又有故意泄漏阴私，隐含挑拔之意。”“放水”徽州方言口语，有意泄漏秘密和个人隐私之谓也。无意泄漏秘密和个人隐私谓之“走水”。《金瓶梅词话》第23回：“干净你这嘴头子，就是个走水的槽，有话到明日不告诉你说了。”

《金瓶梅词话》第58回：“你笑话我老，我那些儿放着老？我半边俏，把你这四个小淫妇儿还不够摆布。”半边俏《金瓶梅词典》释为：“秽语隐指性行为。清，王有光《吴下谚联》卷二《青茄子落坑缸半边俏》条云：青茄色如燕尾，全体光洁，莹然完壁，俏矣。纵遭失落，在于清静之所，上半边之俏可见，即下半边之俏亦未尝没。奈何落于坑缸之中，粪秽之区，过者惜之。”若曰：“尔之俏者，仅此半边矣。”姚灵犀著《瓶外卮言》释为：“陶庵梦忆，谓歪妓也，续金瓶梅四十六回，有云对门河边有的是半边俏，找个陪唱，此应伯爵自道，待考。”“半边俏”，徽州方言口语。徐娘半老，风韵尤存之谓也。年纪大的打扮齐楚整洁谓之“老来俏”。

《金瓶梅词话》第16回：“西门庆家中已盖了两月房屋，三间玩花楼装修将完，只少卷棚还未安磉。”“安磉”《金瓶梅词典》释为：“盖造房子打地基时先安放石础，犹今言奠基。”《瓶外卮言》释为：“即起盖房舍，先安石磉。”以上二种注释错到八国去了。“安磉”，徽州方言口语，查《辞源》“磉”字条注释云：“磉，柱下石也。”徽州建筑属砖木结构，为了防止屋柱直接落地受潮腐烂，柱子下必须安石磉，石磉下还须安磉托，以便承受重压。“磉”，徽州又称“石磉”，“屋柱磉”。形状各异，有圆形、鼓形、方形、六棱形等式样。石磉雕刻精美，花草图案丰富多彩。徽州遗存的明代建筑词堂，民宅均有实物存照。“安磉”，应是在房基建好后，按照屋柱的排例安放石磉。屋架竖好后，还须“理磉”，让屋柱与石磉垂直对齐。

《金瓶梅词话》第76回：“西门庆叫道，怪小油嘴，你怎的不起，叫着他，只不做声推睡。被西门庆双关抱将起来，那春梅从酩子里伸腰，一个鲤鱼打挺，险些儿没把西门庆扫了一交。”“酩子里”，《金瓶梅词典》注释：“暗地里。亦作冥子里。”《野雯曝言》四十四：“众人面面相觑，酩子里只叫的若”。《瓶外卮言》释为：“元代多用

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [56]:
def get_summary(x):
    s = x.split()
    ret = None
    for s0 in s:
        if len(s0) > 300 and ret == None:
            ret = s0
    
    return ret

In [180]:
df['summary'] = df.content.map(lambda x: get_summary(x))

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70995 entries, 0 to 83468
Data columns (total 22 columns):
url             70995 non-null object
title           70995 non-null object
summary         70995 non-null object
content         70995 non-null object
content_seg     70995 non-null object
s_vec           70995 non-null object
domain          70995 non-null object
words_fq        70995 non-null object
w2v_centroid    70995 non-null object
content_len     70995 non-null int64
hz_w2v_sim      70995 non-null float64
hwh_w2v_sim     70995 non-null float64
hzwh_w2v_sim    70995 non-null float64
hzjz_w2v_sim    70995 non-null float64
ycjc_w2v_sim    70995 non-null float64
hzdk_w2v_sim    70995 non-null float64
hpys_w2v_sim    70995 non-null float64
hjhc_w2v_sim    70995 non-null float64
hs_w2v_sim      70995 non-null float64
xalx_w2v_sim    70995 non-null float64
hxyj_w2v_sim    70995 non-null float64
1st_s           70995 non-null object
dtypes: float64(11), int64(1), object(10)
mem

In [181]:
df = df.dropna(subset =['summary'])

In [182]:
df.head()

,url,title,summary,content,content_seg,s_vec,domain,words_fq,w2v_centroid,content_len,...,hzwh_w2v_sim,hzjz_w2v_sim,ycjc_w2v_sim,hzdk_w2v_sim,hpys_w2v_sim,hjhc_w2v_sim,hs_w2v_sim,xalx_w2v_sim,hxyj_w2v_sim,w2v_centroid_s
0,http://cul.anhuinews.com/system/2016/11/28/007...,洪建华夫妇的竹刻人生,谁能想到，洪建华夫妇这对一无学历、二无家传、三无师承的“三无大师”成了省级非物质文化遗产·徽...,谁能想到，洪建华夫妇这对一无学历、二无家传、三无师承的“三无大师”成了省级非物质文化遗产·徽...,谁 能 想到 ， 洪 建华 夫妇 这 对 一 无 学历 、 二 无 家传 、 三无 师承 的...,"{u'竹木': 0.0114285714286, u'洪': 0.0571428571429...",anhuinews.com,"{u'竹木': 0.0114285714286, u'洪': 0.0571428571429...","[-0.0060236, -0.0103621, 0.00755846, -0.000569...",10703,...,0.463720,0.567314,0.573373,0.633446,0.680938,0.424617,0.311315,0.397321,0.291379,"-0.0060236,-0.0103621,0.00755846,-0.000569638,..."
1,http://cul.anhuinews.com/system/2016/11/21/007...,徽州民歌传人操明花校园授艺,“磨米磨麦，磨米磨麦……”随着教室里传来一阵阵悦耳动听的方言民歌声，操明花正兴致勃勃地教同学...,近日，省非物质文化遗产徽州民歌传承人操明花前往休宁县海阳二小、歙县城关小学和徽州区岩寺镇中心...,近日 ， 省非 物质 文化遗产 徽州 民歌 传承 人操 明花 前往 休宁县 海阳 二小 、 ...,"{u'它': 0.0123456790123, u'原生态': 0.012345679012...",anhuinews.com,"{u'它': 0.0123456790123, u'原生态': 0.012345679012...","[-0.0107781, 0.0129855, 0.00413211, 0.00171845...",1264,...,0.565720,0.358653,0.557331,0.468289,0.517698,0.438723,0.306038,0.308822,0.250730,"-0.0107781,0.0129855,0.00413211,0.00171845,-0...."
2,http://cul.anhuinews.com/system/2016/11/03/007...,黄山楹联漫谈,黄山楹联第一人当推明朝诗人余绍祉。他遍历祖国名山大川，见到无数奇峰怪石，却感到抵不上黄山峰林...,黄山楹联第一人当推明朝诗人余绍祉。他遍历祖国名山大川，见到无数奇峰怪石，却感到抵不上黄山峰林...,黄山 楹联 第一 人当 推 明朝 诗人 余绍祉 。 他 遍历 祖国 名山大川 ， 见到 无数...,"{u'气派': 0.0123456790123, u'名句': 0.012345679012...",anhuinews.com,"{u'气派': 0.0123456790123, u'名句': 0.012345679012...","[-0.0120081, -0.0145887, 0.0144744, -0.0120817...",2028,...,0.255002,0.260202,0.269191,0.340901,0.296441,0.235815,0.191698,0.246864,0.182968,"-0.0120081,-0.0145887,0.0144744,-0.0120817,-0...."
3,http://cul.anhuinews.com/system/2016/01/26/007...,徽派雕刻——徽州木雕艺术,徽州民居、家具木板和圆木雕刻艺术的简称。木雕在旧属徽州各县分布之广在全国屈指可数，宅院内的屏...,徽州民居、家具木板和圆木雕刻艺术的简称。木雕在旧属徽州各县分布之广在全国屈指可数，宅院内的屏...,徽州 民居 、 家具 木板 和 圆木 雕刻 艺术 的 简称 。 木雕 在 旧属 徽州 各县 ...,"{u'吉祥图案': 0.0172413793103, u'少数': 0.0172413793...",anhuinews.com,"{u'吉祥图案': 0.0172413793103, u'少数': 0.0172413793...","[-0.0134407, -0.0106611, 0.0100197, -0.0117281...",721,...,0.463754,0.588129,0.565690,0.765616,0.544592,0.325793,0.382593,0.354244,0.263324,"-0.0134407,-0.0106611,0.0100197,-0.0117281,-0...."
4,http://cul.anhuinews.com/system/2016/01/26/007...,徽派雕刻——徽州砖雕艺术,从早期的简单粗犷、朴素的纹样，逐步演变到后期的繁腹、细致华丽的结构，形成了自己独特的艺术风格...,徽州砖雕徽州砖雕的造型，来源于汉画象砖。汉画象砖大都是模印砖坯刻画后入密烧制，嵌窃时再进行修...,徽州 砖雕 徽州 砖雕 的 造型 ， 来源于 汉 画象 砖 。 汉 画象 砖 大都 是 模印...,"{u'一': 0.0106382978723, u'它': 0.0212765957447,...",anhuinews.com,"{u'一': 0.0106382978723, u'它': 0.0212765957447,...","[-0.0147771, -0.0149903, 0.0132143, -0.0146898...",2852,...,0.426712,0.555901,0.473556,0.605832,0.577255,0.312113,0.289483,0.315999,0.203333,"-0.0147771,-0.0149903,0.0132143,-0.0146898,-0...."


In [183]:
sentences = df['summary']

In [184]:
count = Counter(sentences)

In [185]:
top_N = count.most_common(100)

In [74]:
for t in top_N:
    print t[0], t[1], len(t[0])

本站提供旅游信息服务为主的专业网站。通过精心提供景区景点介绍、旅游产品介绍和旅游服务设施介绍（包括：饭店、宾馆、商场和交通线路和站点）为游客和商家提供一个直观的信息交流平台。本站由金桥网络（叶光明）提供技术支持,网站制作和推广服务，请联系洪老师或联系站长：邮箱 17 388
近日，2016年安徽卫视猴年春晚部分嘉宾名单曝光，曾经依靠网络走红、具有超高人气的歌手许嵩也将加盟徽晚精彩献唱。2014年许嵩发行专辑《不如吃茶去》中的开篇曲目《等到烟火清凉》，因只有一句歌词而引发热议，被网友称为最新“神曲”。 13 326
2015年应该是我看戏最多、最密集的一年。算了下一共看了25部戏，平均半个月一部。最忙的时候，一周看四部戏。最累的时候，东奔西跑，连续三天看三部。合肥大剧院的戏几乎都看了，除了《我们一起晃悠的日子》。安徽大剧院只选了几部有名的去看了，错过了何老师的《栀子花开》。 12 381
　　玉兄，前往杭州的路上，若遇见一名叫温莹儿的女子，请代为留住她，她与南宫绫乃闺中密友，此番一块逃婚，并相约在杭州见面。我于路上巧遇此女，并将她携来杭州，但日前她不告而恐是前去寻找南宫绫了，若玉兄得遇她，务请尽速通知官某。 10 333
当2014年还剩下最后一个月，我们盘点下今年合肥城内大的风尚事件时，发现与往年特别不一样的地方：传统意义上的时尚圈层在开始模糊，它们在与非时尚圈不停交叉、互为因果、迭代向前：文艺名流们忙着跨界、摄影愈加草根化、城内电影发行人少有团结、全民爱潮拍、历来只爱广场舞的社区活动开始高大上、市民休闲方式从未有过如此新锐…… 10 463
汪瑞华）中共安徽省第十届委员会第一次全体会议，11月3日选举李锦斌为省委书记，李国英、信长星为省委副书记。现任中共安徽省委常委，省政府副省长、党组成员，省委组织部副部长，省人力资源和社会保障厅厅长、党组书记，安徽行政学院（安徽经济管理学院）院长（兼）。 10 372
“后来，为顾及颜面，在我逃婚后，你还是不得不将女儿下嫁给让她怀孕的富家子。”勤敬冷哼一声，“这件事根本是插曲而已，少把你的野心冠上冠冕堂皇的理由，你原本就有叛变之心，我的逃婚只是让你找到借口，找了人做我的人皮面具，若真有什么风吹草动，大不了有我当你的替死鬼。” 10 390
“我只是想告诉你他没钱了，你爷爷奶奶觉得他每个月赚的钱已经可以让他过得很舒服了，不想帮忙…

In [161]:
for i in range(68000, 70000):
    print df.iloc[i]['title_c'], i
    #print df.iloc[i]['url']

见面的礼仪 68000
过小年 68001
二十五，磨豆腐 68002
腊月二十九祭祖 68003
除夕守岁 68004
春联贺词大全 68005
中国传统风俗礼仪 68006
春节礼仪之喝茶的礼仪 68007
2010年华侨大学招生章程(2) 68008
2010年华侨大学招生章程(4) 68009
美华夏中校12所分校亮相"华夏之星" 尽展才艺 68010
纽约中文才艺学校将举办开放日活动广招新生 68011
澳洲汉语教学机构组织学子举办汉语才艺表演会 68012
2011年华裔青少年才艺大赛在纽约举行(图)(2) 68013
2011年华裔青少年才艺大赛在纽约举行(图)(4) 68014
美国华夏中文学校总校19所分校毕业生告别母校 68015
促菲中高等教育交流 华大教育展将亮相菲律宾 68016
夏季海外华文教师汉语言文化培训班在华大结业 68017
方天兴:大马友族着迷华裔文化 赞赏华裔价值观 68018
华教成“痼疾” 不谈政策难解决大马华教问题  68019
大马董总：过剩国小师资派华小非解决师荒方案  68020
第四届北京澳生毕业晚会在京举行 68021
"回家过节"泰华裔青少年与广西学生共庆泼水节 68022
丹辘省新民中学洎附属小学 68023
丹辘建德学校 68024
彬那吗拉鄢民英学校 68025
三宝颜中华中学洎附小 68026
三宝颜福泉寺观音学校 68027
宿务毓德学校 68028
宿务耶稣会圣心学校 68029
宿务信心培幼学校 68030
宿务东方学院 68031
宿务崇德学校 68032
朗吗倪地中国中学 68033
怡朗华商中学洎附小 68034
描戈律华明中学 68035
描戈律大同中学洎附小、幼儿园 68036
加帛示中山学校 68037
任马兰社华英中学 68038
那牙嘉南中学 68039
黎牙实备中华科技学校 68040
黎牙实备崇德中学 68041
淡描戈培青中学洎附属小学 68042
泰国夜丰颂府政府访问团到访广西华侨学校 68043
广西桂林首家市级华文教育基地揭牌 68044
广西阳朔县高田民族中学成为桂林市华教基地 68045
广西华侨学校与印尼东勿里洞县政府达合作协议 68046
海外华文媒体广西行采访团走进广西华侨学校 68047
培英学校 68048
公立育侨学校 68049
国务院侨办主任裘援平

池州一老人游玩时从高处坠落 头部严重受伤 68375
“财神节”忙烧香 九华山迎来史上最多游客 68376
九华山春节黄金周稳居安徽重点景区人气榜首 68377
池州渔民误捕中华鲟 放归大自然 68378
池州市工商局12315中心 及时处理节日消费问题 68379
九华山风景区假日旅游火爆开局 68380
池州青阳获评全国农村集体“三资”管理示范县 68381
为躲避电子监控 铜陵钓客竟套牌跨市到池州钓鱼 68382
游客九华山丢失钱包 索道员工完璧归赵 68383
青春志愿行 温暖回家路 68384
大场变小场 技工仍吃香 68385
出境旅游大热 东至申请办证人数井喷 68386
全国两会 | 人大代表夏玉发：交通事故赔偿该打破城乡差别 68387
池州市取消停征和免征 61项行政事业性收费 68388
池州公开发布市级政府权力清单 和责任清单目录 68389
九华山羊年“开门红” 春节黄金周央视五次报道景区旅游 68390
九华山旅游“叩开”资本市场大门 68391
央视五次报道九华山旅游 68392
池州市8.4万党员开展集中春训 68393
2014年度综合考核工作全面启动 68394
上岸渔民一家人 新年开启新生活 68395
池州“活雷锋” 一茬又一茬 68396
杏花村里“杏花节” 68397
父子同台共“舞”池州傩 68398
爱心加油站 公益学雷锋 68399
池州三县一区地质灾害防治获评全国先进 68400
疑因池塘水质污染 池州数十村民上吐下泻 68401
贵池正月傩戏活动吸引海内外关注 68402
池州市公务用车制度改革工作动员会召开 68403
池州2015年法律援助惠民实事全面完成 68404
滁州内城河改造一期将完工 68405
马鞍山试用国土资源管理“一张图” 可缩短土地证办理时间 68406
阜阳今年考普通话计划出炉了 全年共四次机会 68407
黄山市“2016就业春风行动”拉开序幕 68408
铜陵市博物馆元旦期间照常试运行 营业时间早九晚四 68409
芜湖今年春运汽车站继续“服务上门” 68410
宣城市“三品一标”位居全省前列 有效用标企业达165家 68411
蚌埠往事风云录：民清时代的蚌埠人口雏形 68412
芜湖爆炸17人死亡 男子看新闻后点打火机试探煤气炸毁自家屋 68413
阜阳三县入选中国民间文化艺术之乡名

葱乡情 68991
妈妈，您在天堂过得好吗？ 68992
我的童年梦圆与人生感悟(2) 68993
易法三章 68994
安徽十大花鼓灯老艺人之一——王传先 68995
通济渠在淮北的历史遗存及深远影响(2) 68996
通济渠在淮北的历史遗存及深远影响(3) 68997
通济渠在淮北的历史遗存及深远影响(4) 68998
凤台县历史大事记(3) 68999
建国前的淮南矿路(2) 69000
罗炳辉将军在淮南抗日民主根据地(9) 69001
中共淮南市委的组织沿革及工作机构 69002
黄梅戏《天仙配》整本（在线视频）(2) 69003
“怀腔”为黄梅戏的正宗腔系 69004
太平新丰王氏探源 69005
徽州建筑的特色 69006
徽州为什么又叫新安 69007
徽州也有位抗倭英雄 69008
安徽齐云山道教音乐初探(3) 69009
民办教育家程小江先生印象记 69010
安徽休宁人物——程家柽 69011
歙县地质历史 69012
歙县新安画派 69013
黟县历史与现状概述 69014
祁门历史上的主要自然灾害(2) 69015
祁门县人口源流 69016
蟹连鱼肚 69017
徽派雕刻——忆千年沧桑，探秘西递牌楼 69018
钟馗造像与汪氏一门——谨以此文纪念一位徽州木雕老者(3) 69019
杂谈徽派建筑“徽”字之妙(组图)(2) 69020
杂谈徽派建筑“徽”字之妙(组图)(3) 69021
试论徽州古建筑超静定文化体系及生命力(4) 69022
查济二甲祠 69023
安徽大学苏式建筑群 见证安徽教育发展 69024
徽商在徽文化创建中的贡献探微 69025
徽商儒家思想根源及对区域经济的影响 69026
捆绑“徽文化”徽商开打翻身战 69027
杏花村实属安徽贵池——“杏花村”地望之争辨析(6) 69028
鲍加 69029
黄学广 69030
朱继胜 69031
李文深 69032
民兵英雄胡国章 69033
刘铭传(2) 69034
徐谦(2) 69035
徐锡麟(2) 69036
李世焱(2) 69037
马琮璜 69038
郑赞丞 69039
宋玉琳 69040
马祥斌 69041
程华亭 69042
张敬尧 69043
张根仁 69044
范鸿仙 69045
周贻春 69046
方兆鳌 69047
程良 69048
郑伯川 69049


安徽省内长江以南地区特色民俗汇总(9) 69795
安徽省内各地的方言故事(2) 69796
阜阳市伊斯兰教的传入和发展 69797
合肥考释(4) 69798
淮上酒家 转瞬即成过眼烟云 69799
格物街:对河成双景 相看两不厌(2) 69800
探古识今 话“庐州文运” 69801
合肥当真没出过一个“状元”？ 69802
李群，这个状元来得好讨巧！ 69803
大城腾飞的文化之魂 69804
勤劳巷 步行街旁的另类繁华 69805
解析“陷巢州”与“涨庐州”链秘 69806
邓小平登黄山的随行摄影师(3) 69807
芜湖县文化馆协办全县法治文艺汇演 69808
是谁凭臆断给明太祖朱元璋定籍凤阳(12) 69809
是谁凭臆断给明太祖朱元璋定籍凤阳(19) 69810
是谁凭臆断给明太祖朱元璋定籍凤阳(25) 69811
是谁凭臆断给明太祖朱元璋定籍凤阳(40) 69812
一副绝妙绝佳的劝民劝父母联 69813
宅坦村观感 69814
问津长干塝(2) 69815
晏八风 69816
为什么我们的学校培养不出杰出人才？(2) 69817
“来苏”非缘迎东坡——绩溪“来苏桥”得名考辩(2) 69818
祁门县明清时期民间民俗碑刻的保护研究(11) 69819
皖旅整改集结号 淮北相山风景区成立整改小组改造景区环境 69820
石维坚做客艺术名家大讲堂——真情比演技更重要 69821
为老叉车厂定格怀旧的味道 合工大学生获设计大奖 69822
合肥：女孩遇“飞车党”抢手机 两男子一旁“看戏” 69823
阜阳警方破获一起网络特大制售假冒品牌服装案 69824
微笑挑战刷屏朋友圈 今天你秀了吗 69825
阜阳小伙两万斤白菜免费送2000多户太原居民 69826
芜湖计划投资25亿元为农民修路架桥 69827
芜湖4家县级公立医院将实施临床路径管理 69828
宣城农业科技园区晋级“国家队” 69829
雅会夏至节 69830
记忆中的外婆桥 69831
岛上书店：没有谁是一座孤岛 每本书都是一个世界 69832
王荣滨《中小学德育生活》一书出版 69833
教育部电子商务行业指导委员会副秘书长王鑫致辞发言稿 69834
阜阳男子手术后丧失性功能 起诉医院获赔68.7万元 69835
安徽破获一制售假酒案 涉案金额千余万元 69836
按照身高穿对裙子 真的能变

In [186]:
def get_clean_title(x):
    s = x.decode('utf-8')
    s = s.replace(u'-中安在线-徽文化', '')
    s = s.replace(u'-徽文化-中安在线', '')
    s = s.replace(u' 中安在线-徽文化频道', '')    
    s = s.replace(u' 徽文化频道 中安在线', '')
    s = s.replace(u'-中国华文教育网', '')    
    s = s.replace(u'中国华文教育网－', '')   
    s = s.replace(u'中国华文教育网-', '')       
    s = s.replace(u'-故园徽州网 - Powered by Discuz!', '')
    s = s.replace(u'婺源论坛 - 江西婺源旅游 - 婺源老家旅游论坛第一门户', '')
    s = s.replace(u'- 醉美婺源旅游网', '')

    s = s.split('_')
    
    if len(s[0]) > 0:
        return s[0]
    else:
        return None

In [187]:
df['title_c'] = df.title.map(lambda x: get_clean_title(x))

In [188]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70995 entries, 0 to 83468
Data columns (total 23 columns):
url               70995 non-null object
title             70995 non-null object
summary           70995 non-null object
content           70995 non-null object
content_seg       70995 non-null object
s_vec             70995 non-null object
domain            70995 non-null object
words_fq          70995 non-null object
w2v_centroid      70995 non-null object
content_len       70995 non-null int64
hz_w2v_sim        70995 non-null float64
hwh_w2v_sim       70995 non-null float64
hzwh_w2v_sim      70995 non-null float64
hzjz_w2v_sim      70995 non-null float64
ycjc_w2v_sim      70995 non-null float64
hzdk_w2v_sim      70995 non-null float64
hpys_w2v_sim      70995 non-null float64
hjhc_w2v_sim      70995 non-null float64
hs_w2v_sim        70995 non-null float64
xalx_w2v_sim      70995 non-null float64
hxyj_w2v_sim      70995 non-null float64
w2v_centroid_s    70995 non-null object
t

In [189]:
def map_badurl(x):
    if 'http://www.hzwh.com/news/1/' in x:
        return None
    else:
        return x

In [190]:
df['url_c'] = df.url.map(lambda x: map_badurl(x))

In [193]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70391 entries, 0 to 83468
Data columns (total 24 columns):
url               70391 non-null object
title             70391 non-null object
summary           70391 non-null object
content           70391 non-null object
content_seg       70391 non-null object
s_vec             70391 non-null object
domain            70391 non-null object
words_fq          70391 non-null object
w2v_centroid      70391 non-null object
content_len       70391 non-null int64
hz_w2v_sim        70391 non-null float64
hwh_w2v_sim       70391 non-null float64
hzwh_w2v_sim      70391 non-null float64
hzjz_w2v_sim      70391 non-null float64
ycjc_w2v_sim      70391 non-null float64
hzdk_w2v_sim      70391 non-null float64
hpys_w2v_sim      70391 non-null float64
hjhc_w2v_sim      70391 non-null float64
hs_w2v_sim        70391 non-null float64
xalx_w2v_sim      70391 non-null float64
hxyj_w2v_sim      70391 non-null float64
w2v_centroid_s    70391 non-null object
t

In [192]:
df = df.dropna(subset =['title_c'])
df = df.dropna(subset =['url_c'])

In [194]:
df['title'] = df['title_c']

In [195]:
def map_serialize(x):
    #print len(x)
    ret = []
    idx = 0
    for v in x:
        #print idx, v
        ret.append(str(v))
        idx += 1
    #print ret
    return ','.join(ret)

In [170]:
#df['w2v_centroid_s'] = df.w2v_centroid.map(lambda x: map_serialize(x))

In [202]:
df_out = df[['url', 'title', 'summary', 'w2v_centroid_s', 'hzwh_w2v_sim', 'hzjz_w2v_sim', 'ycjc_w2v_sim', 'hzdk_w2v_sim',
             'hpys_w2v_sim', 'hjhc_w2v_sim', 'hs_w2v_sim', 'xalx_w2v_sim', 'hxyj_w2v_sim']]

In [206]:
for t in df_out['summary']:
    if '的思想，' in t:
        print t
        print '----------'

到目前为止，徽州文化经济价值的开发与发展还是十分不够的。表现在，既有的开发仅是停留在文化与旅游相结合这一条道路上，这是十分有限和直接的。文化与经济的结合有多种道路选择，存在多种结合的形式，它可以融入许多的产业结构之中，在第一产业、第二产业和第三产业的各个领域，都有自己开发与发展的空间和价值，并且，在文化产业本身上，它前途更是无量。其次，即使是在与旅游相结合的这一条道路，尽管我们已取得了一些成绩，但严格说来，到目前为止，我们的文化旅游事业发展还是初始、简单和低效益的，只是停留在简单观光，徽州文化底蕴和内涵展现与利用仅仅是表象的、少许的一点。第三，再就是徽州文化的观光旅游，甚至到目前为止，也还是不成熟和属自为阶段的，至少是统一的规划与管理没有。景点如散沙，特点不突出，存在单调重复性。没有全市一盘棋的思想，客观上存在相互分立以及互耗的现象，这是亟待解决的。因此，我们要立于新的战略高度，重新审视和规划徽州文化资源的开发潜力与发展道路，切实而高效地将徽州文化资源的开发作为21世纪黄山市经济社会发展的一个重要增长点，发挥其巨大作用。 （作者：刘伯山）
----------
徽州人远祖不少由中原迁宋，长期受中原儒学文化熏陶，特别是程朱理学深入人心的浸润教化，儒学“国家兴亡，匹夫有责”的思想，处庙堂之高则忧其民，处江湖之远则忧其君的思想，先天下之忧而忧，后天下之乐而乐的思想，徽州人身体之、力行之，许多志士胸怀“修身齐家治国平天下”之宏愿，宋以来“名臣辈出，多为御史谏官”，代有伟人，廉吏能臣彪炳史册。
----------
而中华文化能够薪火相传、弦歌不辍的最主要原因，是因为传统礼乐道德文化的特性所致，而礼乐文化的载体，则又与中华民族那源远流长的家族式文化承载体系密不可分的。中国人的特点是聚族而居，中国文化的传承除了国家的倡导外，主要是靠家族文化的力量。本姓本氏的长辈们，把他们平生日积月累的经验、对世界的认识，特别是他们的思想，毫无保留地传授给下一代，成为下一代生存的理念。这也成为中华文化代代相传的一个基本动力。中国人最注重的所谓“孝道”的一个重要内容，所谓“无改于父道”，不过如此。
----------
徽州文化博大精深、源远流长。近些年来，徽学研究不仅在我们徽州“热浪滚滚”，就是世界各地，也已研风同盛，成为一门“显学”。我们徽州先后成立了戴震学术思想研究会、程朱理学研究会和胡适思

In [203]:
df_out.to_csv('../data/clean/huizhou-web-slim_v3.csv', encoding='utf-8', index=False)

In [201]:
def deserialize(x):
    #print x
    ret = []
    s = x.strip('[]')
    vals = s.split(',')
    cnt = 0
    for val in vals:
        #print cnt, val
        #print '-----'
        cnt += 1
        ret.append(float(val))
    return np.asarray(ret)

for i in range(0, len(df)):
    x = df.iloc[i]['w2v_centroid_s']
    print deserialize(x)